# Entraînement Local du Modèle Morningstar

In [1]:
# Installer les dépendances si nécessaire
%pip install tensorflow pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import os

# Définir le chemin absolu du projet
PROJECT_ROOT = Path(os.getcwd()).parent

# Ajouter le répertoire racine au PYTHONPATH
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from model.training.data_loader import load_and_split_data
from model.architecture.morningstar_model import MorningstarModel

2025-04-19 13:03:03.338895: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-19 13:03:03.344505: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-19 13:03:03.432592: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-19 13:03:03.433969: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2025-04-19 13:03:05.029396: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Configuration
DATA_PATH = str(PROJECT_ROOT / 'data' / 'processed' / 'temp_sol_training_local.parquet')
MODEL_SAVE_PATH = str(PROJECT_ROOT / 'model' / 'training' / 'morningstar_local.h5')

# Mapping complet pour les régimes de marché
REGIME_MAPPING = {
    'bearish': 0,
    'bearish_placeholder': 0,
    'sideways': 1,
    'sideways_placeholder': 1,
    'bullish': 2,
    'bullish_placeholder': 2
}

In [4]:
# Chargement des données
(x_train, x_llm_train), y_train = load_and_split_data(
    DATA_PATH,
    label_columns=['trading_signal', 'volatility', 'market_regime', 'level_sl', 'level_tp'],
    label_mappings={'market_regime': REGIME_MAPPING},
    as_tensor=True
)

print(f"Données chargées - X_train: {x_train.shape}, X_llm: {x_llm_train.shape}")
print(f"Labels disponibles: {list(y_train.keys())}")

--- Debug: Data Loader Feature Selection ---
Columns to exclude (labels): ['level_sl', 'level_tp', 'market_regime', 'trading_signal', 'volatility']
LLM columns prefix: 'llm_'
Selected technical features (38):
  First 5: ['open', 'high', 'low', 'close', 'volume']
  Last 5: ['dummy_pipeline_15', 'dummy_pipeline_16', 'dummy_pipeline_17', 'dummy_pipeline_18', 'dummy_pipeline_19']
--- End Debug ---
Colonnes techniques détectées (38):
1. ATR
2. BBL
3. BBM
4. BBU
5. EMA_long
6. EMA_short
7. MACD
8. MACDh
9. MACDs
10. RSI
11. SMA_long
12. SMA_short
13. STOCHd
14. STOCHk
15. close
16. dummy_pipeline_15
17. dummy_pipeline_16
18. dummy_pipeline_17
19. dummy_pipeline_18
20. dummy_pipeline_19
21. dummy_placeholder_1
22. dummy_placeholder_10
23. dummy_placeholder_11
24. dummy_placeholder_12
25. dummy_placeholder_13
26. dummy_placeholder_14
27. dummy_placeholder_2
28. dummy_placeholder_3
29. dummy_placeholder_4
30. dummy_placeholder_5
31. dummy_placeholder_6
32. dummy_placeholder_7
33. dummy_placehol

Données chargées - X_train: (8783, 38), X_llm: (8783, 768)
Labels disponibles: ['trading_signal', 'volatility', 'market_regime', 'level_sl', 'level_tp']


In [5]:
# Initialisation du modèle
model = MorningstarModel()
model.initialize_model()

# Configuration de compilation
losses = {
    'signal': 'sparse_categorical_crossentropy',
    'volatility_quantiles': 'mse',
    'market_regime': 'sparse_categorical_crossentropy',
    'sl_tp': 'mse'
}

metrics = {
    'signal': ['accuracy'],
    'volatility_quantiles': ['mae'],
    'market_regime': ['accuracy'],
    'sl_tp': ['mae']
}

model.model.compile(
    optimizer='adam',
    loss=losses,
    metrics=metrics
)

In [6]:
# Entraînement
history = model.model.fit(
    {'technical_input': x_train, 'llm_input': x_llm_train},
    {'signal': y_train['trading_signal'], 
     'volatility_quantiles': y_train['volatility'],
     'market_regime': y_train['market_regime'],
     'sl_tp': tf.stack([y_train['level_sl'], y_train['level_tp']], axis=1)},
    epochs=50,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/50


  1/220 [..............................] - ETA: 13:20 - loss: 12975.6641 - signal_loss: 1.5222 - volatility_quantiles_loss: 1.9225 - market_regime_loss: 2.0680 - sl_tp_loss: 12970.1504 - signal_accuracy: 0.3125 - volatility_quantiles_mae: 1.1100 - market_regime_accuracy: 0.1875 - sl_tp_mae: 112.5385

 11/220 [>.............................] - ETA: 1s - loss: 12278.0010 - signal_loss: 1.2722 - volatility_quantiles_loss: 2.0574 - market_regime_loss: 1.7289 - sl_tp_loss: 12272.9434 - signal_accuracy: 0.4716 - volatility_quantiles_mae: 1.0381 - market_regime_accuracy: 0.3182 - sl_tp_mae: 109.2530   

 21/220 [=>............................] - ETA: 1s - loss: 11951.8867 - signal_loss: 0.9786 - volatility_quantiles_loss: 2.1454 - market_regime_loss: 1.3769 - sl_tp_loss: 11947.3848 - signal_accuracy: 0.6012 - volatility_quantiles_mae: 1.0492 - market_regime_accuracy: 0.4836 - sl_tp_mae: 107.7116

 31/220 [===>..........................] - ETA: 0s - loss: 11633.0869 - signal_loss: 0.7636 - volatility_quantiles_loss: 1.8906 - market_regime_loss: 1.0531 - sl_tp_loss: 11629.3799 - signal_accuracy: 0.7117 - volatility_quantiles_mae: 0.9747 - market_regime_accuracy: 0.6119 - sl_tp_mae: 106.1972

...

181/220 [=======================>......] - ETA: 0s - loss: 33.2182 - signal_loss: 0.0000e+00 - volatility_quantiles_loss: 0.0018 - market_regime_loss: 0.0000e+00 - sl_tp_loss: 33.2165 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.0325 - market_regime_accuracy: 1.0000 - sl_tp_mae: 4.4742

191/220 [=========================>....] - ETA: 0s - loss: 33.1826 - signal_loss: 0.0000e+00 - volatility_quantiles_loss: 0.0018 - market_regime_loss: 0.0000e+00 - sl_tp_loss: 33.1809 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.0324 - market_regime_accuracy: 1.0000 - sl_tp_mae: 4.4637

202/220 [==========================>...] - ETA: 0s - loss: 33.1843 - signal_loss: 0.0000e+00 - volatility_quantiles_loss: 0.0018 - market_regime_loss: 0.0000e+00 - sl_tp_loss: 33.1825 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.0323 - market_regime_accuracy: 1.0000 - sl_tp_mae: 4.4574

212/220 [===========================>..] - ETA: 0s - loss: 33.1862 - signal_loss: 0.0000e+00 - volatility_quantiles_loss: 0.0018 - market_regime_loss: 0.0000e+00 - sl_tp_loss: 33.1845 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.0324 - market_regime_accuracy: 1.0000 - sl_tp_mae: 4.4629

220/220 [==============================] - 1s 6ms/step - loss: 33.2578 - signal_loss: 0.0000e+00 - volatility_quantiles_loss: 0.0018 - market_regime_loss: 0.0000e+00 - sl_tp_loss: 33.2560 - signal_accuracy: 1.0000 - volatility_quantiles_mae: 0.0324 - market_regime_accuracy: 1.0000 - sl_tp_mae: 4.4640 - val_loss: 4114.0850 - val_signal_loss: 0.0000e+00 - val_volatility_quantiles_loss: 0.3214 - val_market_regime_loss: 0.0000e+00 - val_sl_tp_loss: 4113.7642 - val_signal_accuracy: 1.0000 - val_volatility_quantiles_mae: 0.1405 - val_market_regime_accuracy: 1.0000 - val_sl_tp_mae: 40.4492


In [7]:
# Sauvegarde
Path(MODEL_SAVE_PATH).parent.mkdir(parents=True, exist_ok=True)
model.model.save(MODEL_SAVE_PATH)
print(f"Modèle sauvegardé à {MODEL_SAVE_PATH}")

Modèle sauvegardé à /home/morningstar/Desktop/crypto_robot/Morningstar/model/training/morningstar_local.h5
